## Using the BISON API
The USGS provides an API for accessing species observation data. https://bison.usgs.gov/doc/api.jsp

This API is much better documented than the NWIS API, and we'll use it to dig a bit deeper into how the `requests` package can faciliate data access via APIs. 

* We'll begin by replicating the example API call they show on their web page:<br> 
[https://bison.usgs.gov/api/search.json?species=Bison bison&type=scientific_name&start=0&count=1](
https://bison.usgs.gov/api/search.json?species=Bison%20bison&type=scientific_name&start=0&count=1)

In [1]:
#First, import the wonderful requests module
import requests

* Now, we'll deconstruct the example URL into the service URL and parameters, saving the paramters as a dictionary. Note we are just providing a few of the parameters available through the [API](https://bison.usgs.gov/doc/api.jsp#opensearch). We could add more search criteria if we wanted, but for now we just want to grab the first 500 Bison records. 

In [2]:
# Construct the service URL as two components: the service URL and the request parameters
url = 'http://bison.usgs.gov/api/search.json'
params = {'species':'Bison bison',
          'type':'scientific_name',
          'start':'0',
          'count':'500'
         }

* With the components set as variables, we use the `requests.get()` function to send our request off to the server at the address provided, storing the servers response as a variable called `response`. 

In [3]:
#Send the request to the server and store the response as a variable
response = requests.get(url,params)

* This response object contains a number of properties and methods. Let's have a look at the reponse in raw text format. 

In [4]:
#View the reponse in text format
response.text

'{"occurrences":{"legend":{"fossil":359,"observation":927,"centroid":1,"specimen":946,"unknown":6}},"total":2238,"searchTime":0,"offset":0,"data":[{"geo":"Yes","provider":"Conservation of Arctic Flora and Fauna","name":"Bison bison","decimalLongitude":"-117.6500015258789","decimalLatitude":"61.366668701171875","occurrenceID":"68428522","common_name":"bison, American bison, American Bison, Bisonte americano","basis":"Observation"},{"geo":"Yes","provider":"Conservation of Arctic Flora and Fauna","name":"Bison bison","decimalLongitude":"-117.6500015258789","decimalLatitude":"61.366668701171875","occurrenceID":"68428523","common_name":"bison, American bison, American Bison, Bisonte americano","basis":"Observation"},{"geo":"Yes","provider":"Conservation of Arctic Flora and Fauna","name":"Bison bison","decimalLongitude":"-117.6500015258789","decimalLatitude":"61.366668701171875","occurrenceID":"68428524","common_name":"bison, American bison, American Bison, Bisonte americano","basis":"Observ

**Yikes**, that's much less readable than the NWIS output!

Well, that's because the response from the BISON server is in **JSON** format. JSON, short for *JavaScript Object Notation*, is a text document that stores information in `key`:`value` pairs, *much like a Python dictionary*. Still, it's a raw text object, but one that we convert into a Python dictionary using Python's json package.

In [4]:
#Import the module
import json

#Convert the response 
data = json.loads(response.text)
type(data)

dict

> *Note*: we could also convert this to JSON using the `json` function of the `response` object...<br>The code below has the exact same results as the one above. 

In [5]:
data=response.json()
type(data)

dict

* Ok, if it's a dictionary, what are it's keys? 

In [7]:
#List the keys in the returned JSON object
data.keys()

dict_keys(['occurrences', 'total', 'searchTime', 'offset', 'data', 'species', 'eezs', 'itemsPerPage', 'counties', 'type', 'georeferenced', 'states'])

* What are the values linked with the 'data' key?

In [9]:
#Show the value associated with the `data` key
len(data['data'])

500

* Oh, it's a list of occurrences! Let's examine the first one...

In [10]:
#Display the first "data" value
data['data'][0]

{'geo': 'Yes',
 'provider': 'Conservation of Arctic Flora and Fauna',
 'name': 'Bison bison',
 'decimalLongitude': '-117.6500015258789',
 'decimalLatitude': '61.366668701171875',
 'occurrenceID': '68428522',
 'common_name': 'bison, American bison, American Bison, Bisonte americano',
 'basis': 'Observation'}

* We see it's a dictionary too! Let's list the `decimalLatitude` item value...

In [11]:
#We can get the latitude of the record from it's `decimalLatitude` key
data['data'][0]['decimalLatitude']

'61.366668701171875'

► **So** we see the Bison observations are stored as list of dictionaries which are accessed within the `data` key in the results dictionary generated from the JSON response to our API request. (Phew!)

* With a bit more code we can loop through all the data records and print out the lat and long coordinates...

In [8]:
#Loop thorough each observation and print the lat and long values
for observation in data['data']:
    print (observation['decimalLatitude'],observation['decimalLongitude'])

61.366668701171875 -117.6500015258789
61.366668701171875 -117.6500015258789
61.366668701171875 -117.6500015258789
61.366668701171875 -117.6500015258789
61.366668701171875 -117.6500015258789
61.366668701171875 -117.6500015258789
61.366668701171875 -117.6500015258789
61.366668701171875 -117.6500015258789


KeyError: 'decimalLatitude'

► *If the above throws an error, can you debug it? HINT: the `geo` tag indicates whether coordinate info exist for the record...*

### [Another] Preview of 'Pandas' - that clever Python package with many uses!
Pandas can create a "data frame" from dictionary values. We'll talk about this soon, but can be quite useful!

In [9]:
import pandas as pd
df = pd.DataFrame(data['data'])
df.head()

,basis,common_name,decimalLatitude,decimalLongitude,geo,name,occurrenceID,provider
0,Observation,"bison, American bison, American Bison, Bisonte...",61.366668701171875,-117.6500015258789,Yes,Bison bison,68428522,Conservation of Arctic Flora and Fauna
1,Observation,"bison, American bison, American Bison, Bisonte...",61.366668701171875,-117.6500015258789,Yes,Bison bison,68428523,Conservation of Arctic Flora and Fauna
2,Observation,"bison, American bison, American Bison, Bisonte...",61.366668701171875,-117.6500015258789,Yes,Bison bison,68428524,Conservation of Arctic Flora and Fauna
3,Observation,"bison, American bison, American Bison, Bisonte...",61.366668701171875,-117.6500015258789,Yes,Bison bison,68428525,Conservation of Arctic Flora and Fauna
4,Observation,"bison, American bison, American Bison, Bisonte...",61.366668701171875,-117.6500015258789,Yes,Bison bison,68428526,Conservation of Arctic Flora and Fauna


And Pandas allows us to do some nifty analyses, including subsetting records for a specific provider.
* First we'll get a list of unique providers found in the data

In [10]:
#Generate a list of providers
df.provider.unique()

array(['Conservation of Arctic Flora and Fauna',
       'Berkeley Natural History Museums', 'Royal Ontario Museum',
       'National Museum of Natural History, Smithsonian Institution',
       'Western Australian Museum', 'Australian Museum',
       'University of Texas at El Paso Biodiversity Collections',
       'University of Alaska Museum of the North',
       'University of Colorado Museum of Natural History',
       'Denver Museum of Nature & Science',
       'University of Kansas Biodiversity Institute',
       'Museum of Comparative Zoology, Harvard University',
       'Cornell Lab of Ornithology',
       'Museum of Natural and Cultural History - University of Oregon',
       'Museum of Southwestern Biology',
       'Museo Argentino de Ciencias Naturales',
       'University of Arkansas Collections Facility, UAFMC',
       'University of Alberta Museums',
       'James R. Slater Museum of Natural History',
       'University of Iowa Museum of Natural History',
       'Californi

* Now, we'll subset the rows that include that provider...

In [11]:
df.query("provider == 'Denver Museum of Nature & Science'")

,basis,common_name,decimalLatitude,decimalLongitude,geo,name,occurrenceID,provider
46,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019528,Denver Museum of Nature & Science
47,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019529,Denver Museum of Nature & Science
52,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019534,Denver Museum of Nature & Science
54,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019536,Denver Museum of Nature & Science
58,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019540,Denver Museum of Nature & Science
61,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019543,Denver Museum of Nature & Science
62,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019544,Denver Museum of Nature & Science
65,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019547,Denver Museum of Nature & Science
66,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019548,Denver Museum of Nature & Science
78,Specimen,"bison, American bison, American Bison, Bisonte...",NaN,NaN,No,Bison bison,81019560,Denver Museum of Nature & Science


## Exercise:
* Extract the first 500 red wolf (*"Canis rufus"*) records from the BISON API. 
* Can you create a table listing the records collected by the `University of Kansas Biodiversity Institute`?
* *Challenge*: Can you create a table listing all the records collected in North Carolina?

In [3]:
# Construct the service URL as two components: the service URL and the request parameters

import requests

url = 'http://bison.usgs.gov/api/search.json'
params = {'species':'Canis rufus',
          'type':'scientific_name',
          'start':'0',
          'count':'1000',
          'state':'North Carolina'
         }

In [4]:
#Send the request to the server and store the response as a variable
responsewolf = requests.get(url,params)

In [17]:
responsewolf.text

'{"occurrences":{"legend":{"fossil":23,"observation":1,"centroid":1,"specimen":600,"unknown":37}},"total":698,"searchTime":1160,"offset":0,"data":[{"geo":"No","provider":"National Museum of Natural History, Smithsonian Institution","name":"Canis rufus","occurrenceID":"82909859","common_name":"","basis":"Specimen"},{"geo":"No","provider":"National Museum of Natural History, Smithsonian Institution","name":"Canis rufus","occurrenceID":"82910676","common_name":"","basis":"Specimen"},{"geo":"No","provider":"National Museum of Natural History, Smithsonian Institution","name":"Canis rufus","occurrenceID":"82911142","common_name":"","basis":"Specimen"},{"geo":"No","provider":"National Museum of Natural History, Smithsonian Institution","name":"Canis rufus","occurrenceID":"82911261","common_name":"","basis":"Specimen"},{"geo":"No","provider":"National Museum of Natural History, Smithsonian Institution","name":"Canis rufus","occurrenceID":"82912752","common_name":"","basis":"Specimen"},{"geo":"

In [5]:
#Import the module
import json

#Convert the response 
datawolf = json.loads(responsewolf.text)
type(datawolf)


dict

In [9]:
import pandas as pd
datawolf = json.loads(responsewolf.text)
datawolf['data']
NCdf = pd.DataFrame(datawolf['data'])
NCdf

,basis,common_name,decimalLatitude,decimalLongitude,geo,name,occurrenceID,provider
0,Specimen,,35.5756721496582,-76.29776000976562,Yes,Canis rufus,303013045,James R. Slater Museum of Natural History
1,Specimen,,35.83486557006836,-75.91395568847656,Yes,Canis rufus,306819508,James R. Slater Museum of Natural History
2,Specimen,,35.83486557006836,-75.91395568847656,Yes,Canis rufus,306819509,James R. Slater Museum of Natural History
3,None,,35.11817932128906,-77.0825424194336,Yes,Canis rufus,2108627,BISON
4,None,,35.4823112487793,-76.8420181274414,Yes,Canis rufus,2108762,BISON
5,None,,35.40815734863281,-76.1536865234375,Yes,Canis rufus,2108769,BISON
6,None,,35.40815734863281,-76.1536865234375,Yes,Canis rufus,2108801,BISON
7,None,,36.06132888793945,-76.96236419677734,Yes,Canis rufus,2108807,BISON
8,None,,35.4823112487793,-76.8420181274414,Yes,Canis rufus,2108913,BISON
9,None,,35.4823112487793,-76.8420181274414,Yes,Canis rufus,2108973,BISON
